In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
import pickle

# Identification of the contour belonging to imaging phantoms
## Training of Machine Learning classifiers.



Reading in data:

In [3]:
# Path to CSV of contour endpoints, created using opencv_phantom_detector_test.py
df = pd.read_csv("")

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
df_only_phantoms = df[df["is_phantom"]==1]

Save only phantom contours to a CSV to find patterns by manual inspection.

In [ ]:
df_only_phantoms.to_csv("all_phantoms_from_framingham_folder.csv")

In [ ]:
value = "mean_val"
mean = df_only_phantoms[value].mean()
max = df_only_phantoms[value].max()
min = df_only_phantoms[value].min()
print("{}: Mean: {}  Max: {}  Min:{}".format(value, mean, max, min))

Add means for the three parts of the phantom. Forgot to add that data to the get_endpoints method upon creation of the dataframes, so have to create that manually now.

Creating X and y for training ML models

In [ ]:
y =df["is_phantom"].fillna(0).to_numpy()
X = df.drop(["Unnamed: 0", "Unnamed: 0.1", "hierarchy", "file", "moments", "centroid", "min_loc", "max_loc", "is_phantom"], "columns", inplace=False)

#### Inspect data

In [ ]:
print(df.shape)
df.head()

In [ ]:
print(X.shape)
X.head()

In [ ]:
X = X.to_numpy()

In [ ]:
y

## Create and train logistic regression model

This will be able to tell, whether a given contour is a phantom or not

In [ ]:
log_reg_model = LogisticRegression(random_state=0, max_iter=100).fit(X, y)
log_reg_model.score(X, y)

Test model on any row

In [ ]:
row = 76736 #This one is in fact a phantom, but we do have trouble finding it
true_contour = X[row]
log_reg_model.predict_proba(true_contour.reshape(1,-1))

#### Save model

In [ ]:
filename="logistic_regression_model_for_contour_classification_if_phantom.sav"
pickle.dump(log_reg_model, open(filename, "wb"))

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()#max_depth=3, random_state=0)
rfc.fit(X, y)

In [ ]:
print(rfc.score(X, y))

In [ ]:
row = 76736 #This one is in fact a phantom, but we do have trouble finding it
true_contour = X[row]
rfc.predict_proba(true_contour.reshape(1,-1))

In [ ]:
list = true_contour.reshape(1,-1).tolist()
print(list)

In [ ]:
rfc.predict_proba(true_contour.reshape(1,-1))

So far, random forest seams to work better for our binary classification. That also seems intuitive, with the kind of shape parameters we have.

It might be overfitting to our sample data though, without limiting depth. That remains to be seen, however.

In [ ]:
filename="random_forest_model_for_contour_classification_if_phantom.sav"
pickle.dump(rfc, open(filename, "wb"))

In [ ]:
#loaded_model = pickle.load(open(filename, "rb"))
#loaded_model.predict_proba(true_contour.reshape(1,-1))

## Binary decision tree

Advantage: will hopefully be faster.

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier().fit(X, y)

In [ ]:
clf.predict(X)

In [ ]:
clf.score(X, y)

In [ ]:
filename="decisiontree_model_for_contour_classification_if_phantom.sav"
pickle.dump(clf, open(filename, "wb"))

In [ ]:
import graphviz

dot_data=tree.export_graphviz(clf, out_file=None)
graph=graphviz.Source(dot_data)
graph.render("Graph.pdf")
